![](https://img.kaikeba.com/70350130700202jusm.png)

# AI核心课作业2-3-用决策树对优惠券使用进行预测

# 导入读取数据的工具包

In [1]:
import pandas as pd
import numpy as np

# 读取数据集
数据放在/data/course_data/AI路径下,我们先读取数据

In [2]:
train_data = pd.read_csv('/data/course_data/AI/ccf_offline_stage1_train.csv')

# 查看数据情况
读取数据后，一般首先看一下数据的基本情况，这样我们能够对数据有一个整体的了解

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   User_id        int64  
 1   Merchant_id    int64  
 2   Coupon_id      float64
 3   Discount_rate  object 
 4   Distance       float64
 5   Date_received  float64
 6   Date           float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


# 查看前5条样本

In [4]:
train_data.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [5]:
print(train_data.shape)

(1754884, 7)


# 问题一：丢弃带有缺失值的数据

In [8]:
######## your code ~ 1 line ########
# TODO:删除带有缺失值的行和列,请在此行填写答案
train_data = train_data.dropna(how='any')
######## your code end ########

In [9]:
print(train_data.shape)# 打印删除缺失数据后的数据大小

(67165, 7)


知识点1：数据预处理经常涉及到的一个问题就是缺失值的处理问题，关于缺失值的处理方法，可以参考这篇文章：https://blog.csdn.net/weixin_44481878/article/details/88987662  
知识点2：pandas里面删除缺失数据的高频函数dropna(),详细介绍请参考下面的文章：https://zhuanlan.zhihu.com/p/108114844

### Discount_rate是object类型的，object在pandas中代表字符串，字符串类型不能输入模型中，所以需要改为数值类型

In [10]:
# 查看Discount_rate特征的唯一值
# [0,1] 表示折扣率
# x:y 表示满 x 减 y
print('Discount_rate 类型：\n',train_data['Discount_rate'].unique())

Discount_rate 类型：
 ['20:1' '20:5' '30:5' '50:10' '10:5' '50:20' '100:10' '30:10' '50:5'
 '30:1' '100:30' '0.8' '200:30' '100:20' '10:1' '200:20' '0.95' '5:1'
 '100:5' '100:50' '50:1' '20:10' '150:10' '0.9' '200:50' '150:20' '150:50'
 '200:5' '300:30' '100:1' '200:10' '150:30' '0.85' '0.6' '0.5' '300:20'
 '200:100' '300:50' '150:5' '300:10' '0.75' '0.7' '30:20' '50:30']


### 打折类型
### x:y 表示满 x 减 y，          将 x:y 类型的字符串设为1
### [0,1] 表示折扣率，           将 [0,1] 类型的字符串设为 0

In [11]:
# 定义转换Discount_rate特征的函数
def getDiscountType(row):
    if ':' in row:
        return 1
    else:
        return 0

# 问题二：将 Discount_rate 转化为数值特征

In [16]:
######## your code ~ 1 line ########
# TODO:将Discount_rate特征的数据转换为数值型特征，使用我们自己定义的getDiscountType+apply函数
train_data = train_data.apply(lambda x : getDiscountType(x) if x.name=='Discount_rate' else x)
######## your code end ########

知识点3：pandas里面apply函数的使用可以参考这篇文章：https://www.jianshu.com/p/4fdd6eee1b06?from=singlemessage

In [18]:
train_data.info()# 打印将Discount_rate特征转换为数值型特征后，数据的基本信息

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67165 entries, 6 to 1754880
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   User_id        67165 non-null  int64  
 1   Merchant_id    67165 non-null  int64  
 2   Coupon_id      67165 non-null  float64
 3   Discount_rate  67165 non-null  int64  
 4   Distance       67165 non-null  float64
 5   Date_received  67165 non-null  float64
 6   Date           67165 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 6.6 MB


# 导入模型，划分数据集的包

In [19]:
# 导入DecisionTreeClassifier模型
from sklearn.tree import DecisionTreeClassifier
# 导入 train_test_split，用于划分数据集和测试集
from sklearn.model_selection import train_test_split
# 导入 accuracy_score 评价指标
from sklearn.metrics import accuracy_score

In [21]:
train_data

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
6,1439408,2632,8591.0,0,0.0,20160516.0,20160613.0
33,1113008,1361,11166.0,0,0.0,20160515.0,20160521.0
38,2881376,8390,7531.0,0,0.0,20160321.0,20160329.0
69,114747,6901,2366.0,0,0.0,20160523.0,20160605.0
75,114747,5341,111.0,0,0.0,20160127.0,20160221.0
...,...,...,...,...,...,...,...
1754833,1437872,7706,416.0,0,4.0,20160129.0,20160202.0
1754873,212662,2934,5686.0,0,2.0,20160321.0,20160330.0
1754877,212662,3021,3739.0,0,6.0,20160504.0,20160508.0
1754878,212662,2934,5686.0,0,2.0,20160321.0,20160322.0


# 问题三：为数据集添加一个label列

### 标注标签Label，标注哪些样本是正样本 y=1，哪些是负样本 y = 0
### 预测目标：用户在领取优惠券之后 15 之内的消费情况
### (Date - Date_received <= 15) 表示领取优惠券且在15天内使用，即正样本，y = 1
### (Date - Date_received > 15)   表示领取优惠券未在15天内使用，即负样本，y = 0

In [31]:
# 定义样本标签函数
def label(row):
    ######## your code ~ 1 line ########
    # TODO：计算用户消费日期和领取优惠券的时间间隔
    td = pd.to_datetime(int(row['Date']),format='%Y%m%d') - pd.to_datetime(int(row['Date_received']),format='%Y%m%d') 
    ######## your code end ########
    
    if td <= pd.Timedelta(15, 'D'): # 如果消费日期和领取优惠券的时间间隔小于等于15，则标签设置为1
        return 1
    else:
        return 0

train_data['label'] = train_data.apply(label, axis=1)

知识点4：pandas关于时间的教程https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

In [32]:
# 统计正负样本的分布
print(train_data['label'].value_counts())

1    57060
0    10105
Name: label, dtype: int64


Tips:当正负样本比例差距过大时（比如正负样本比例为1:10时），此时的数据集称为非平衡数据集，非平衡数据集会影响我们的模型，
因此需要对数据集进行处理，方法包括：正采样、负采样、数据合成等，具体方法可以参考这篇文章：
https://blog.csdn.net/a8039974/article/details/83684841

# 问题四：对数据集进行划分：80%训练集 、20%测试集

In [37]:
# 划分训练集和验证集，比例分别为0.8、0.2
# TODO：请在此位置填写划分训练集和验证集的函数
X_train, X_test, y_train, y_test = train_test_split(train_data[["User_id","Merchant_id","Coupon_id","Date_received","Discount_rate","Distance"]],
                                    train_data["label"],
                                    test_size = 0.2,
                                    random_state = 7)

知识点5：在sklearn中划分数据集有多种方法，比如当采用k折交叉验证时使用KFold函数；当采用留一法时使用LeaveOneOut函数；
更多的数据集划分方法可以参考这篇文章：https://www.cnblogs.com/cmybky/p/11772655.html                      
知识点6:在模型中，我们使用了"User_id"等6个特征，我们使用的6个特征是否对预测我们的label都有意义呢？我们是否能够构造更多有效的特征来纳入模型，以提高我们模型的预测的效果呢？希望同学们能多多思考、多多探索！

In [38]:
# 查验训练样本的数量和类别分布
y_train.value_counts()

1    45656
0     8076
Name: label, dtype: int64

In [39]:
# 查验测试样本的数量和类别分布
y_test.value_counts()

1    11404
0     2029
Name: label, dtype: int64

# 初始化分类决策树模型， 树的深度为5层

In [40]:
model = DecisionTreeClassifier(max_depth=5, random_state=1)

sklearn中决策树的使用可以参考sklearn官方文档里面对于决策树的介绍：https://scikit-learn.org/stable/modules/tree.html

# 模型训练

In [41]:
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

# 问题五：模型预测

In [42]:
######## your code ~ 1 line ########
# TODO:请在此行填写答案，使用训练好的模型对测试数据进行预测
y_pred = model.predict(X_test)
######## your code end ########

# 模型评估

In [43]:
accuracy_score(y_test, y_pred)

0.8523784709297998

# 将模型选择特征的标准改为entropy 

In [44]:
model = DecisionTreeClassifier(criterion='entropy', random_state=1, max_depth=5)

# 模型训练

In [45]:
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

# 问题五：模型预测

In [46]:
######## your code ~ 1 line ########
# TODO:请在此行填写答案，使用训练好的模型对测试数据进行预测
y_pred = model.predict(X_test)
######## your code end ########

# 评估

In [47]:
accuracy_score(y_test, y_pred)

0.8520062532569046

### 除以上关键步骤，同学也可以自行对数据进行探索，以及其他任何形式对特征预处理方法和特征工程处理。希望同学们能够通过此次作业，着重理解机器学习任务的开展流程，后续我们会讲到更多提升模型效果的方法。对于数据处理的技能和方法，鼓励大家课下多投入时间去探索。祝大家学习进步！

![image alt <](http://5b0988e595225.cdn.sohucs.com/images/20190420/1d1070881fd540db817b2a3bdd967f37.gif)